In [ ]:
from UtilDatasetPrepFromLORE import *
from Rankers import *
from PreferentialSampling import *
from UniformSampling import *
from StandardEval import *
from FairnessEval import *
from Explainers import *

# To Set 

## Sampling

In [ ]:
filename = 'german_credit.csv'
path_res = 'Experiments/Res-Test/'
path_data = 'datasets/'
sensitive = ['foreign_worker'] #OR personal_status_sex
discrete, df = prepare_german_dataset(filename, path_data, sensitive)
target = 'target'
column_names = df.columns.values.tolist()
column_names.remove(target)
class_names=['Good','Bad']
adClass=0
disClass=1
protected = ['foreign_worker_no','foreign_worker_yes']
protected_feature='foreigners' 
non_sensitive = [elem for elem in column_names if elem not in protected]
adAttr=None
disAttr=None

### Uniform Sampling

In [ ]:
samplingName = "US_"
X,Y,X_blind,X_train,X_test,y_train,y_test,df=prepare_for_sampling(df,protected)
'''X_test = pd.DataFrame(X_test, columns=column_names)
X_test = X_test.sample(300)
y_test = y_test[X_test.index]
X_test = X_test.values'''
findFreq(target,protected,df)

In [ ]:
df_new=UniformSampling(target,protected,df,adClass,disClass,adAttr,disAttr)
findFreq(target,protected,df_new)
X_train,X_train_blind,X_train_mod,X_train_mod_blind,y_train_mod,X_test,X_test_blind=prepare_for_classification(df_new,X_train,X_test,column_names,protected,False)

### Preferential Sampling

In [ ]:
samplingName = "PS_"
'''X = X.sample(3000)
X_blind = X_blind.iloc[X.index]
Y = Y[X.index]'''
blind,ranker = rankers(X,X_blind,Y)

In [ ]:
PIK = "Ranker.dat"
with open(path_res+"/"+PIK, "wb") as f:
  pickle.dump(ranker, f)

In [ ]:
PIK = "Ranker.dat"
with open(path_res+"/"+PIK, "rb") as f:
  ranker = pickle.load(f)

In [ ]:
df_new=PreferentialSampling(target,protected,ranker,blind,df,adClass,disClass,adAttr,disAttr)
findFreq(target,protected,df_new)
X_train,X_train_blind,X_train_mod,X_train_mod_blind,y_train_mod,X_test,X_test_blind=prepare_for_classification(df_new,X_train,X_test,column_names,protected,True)

## Clfs

In [ ]:
clfName = "_MLP_"
clf = MLPClassifier()
param_grid = dict(hidden_layer_sizes=[(50, 50, 50), (50, 100, 50)],
                          activation=['tanh', 'relu'],
                          solver=['sgd', 'adam'],
                          alpha=[0.0001, 0.05],
                          learning_rate=['constant', 'adaptive']
                          )

In [ ]:
clfName = "_RF_"
clf = RandomForestClassifier()
param_grid = {'n_estimators': [200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth' : [4,5,6,7,8], 'criterion' : ['gini', 'entropy'], 'bootstrap' : [True, False]}

## Explainer & Reference Set 

In [ ]:
XAISet = "_Fix"

In [ ]:
XAISet = "_Change"

In [ ]:
explainerName = "_SHAP_"

In [ ]:
explainerName = "_LIME_"

# DEF

## Performance

In [ ]:
clfOrig,clfOrigBlind,clfMit,clfMitBlind,res = Classifiers(clf,param_grid,target,column_names,X_train,X_train_mod,X_train_blind,X_train_mod_blind,y_train,y_train_mod,X_test,X_test_blind,y_test,protected,adClass,disClass,adAttr,disAttr)
res = res.round(3)
res.to_csv(path_res+samplingName+clfName+'Performances.csv')
res

In [ ]:
PIK = samplingName+clfName+".dat"
with open(path_res+"/"+PIK, "wb") as f:
  pickle.dump([clfOrig,clfOrigBlind,clfMit,clfMitBlind], f)

In [ ]:
PIK = samplingName+clfName+".dat"
with open(path_res+"/"+PIK, "rb") as f:
  clfOrig,clfOrigBlind,clfMit,clfMitBlind = pickle.load(f)
clfs={'clfOrig':clfOrig,'clfOrigBlind':clfOrigBlind,'clfMit':clfMit,'clfMitBlind':clfMitBlind}

## Fairness

In [ ]:
plotDisc(res,path_res,samplingName+clfName)

In [ ]:
FATInput=DataDescription(X_test,y_test,class_names,column_names,protected,protected_feature)

In [ ]:
resDisparateImp=disparateImpactMetrics(clfs,FATInput,X_test,X_test_blind,y_test,protected_feature)
resDisparateImp

In [ ]:
standardMetricsPerGroup(clfs,column_names,target,X_test,y_test,protected_feature,protected)

## Explainability

In [ ]:
if XAISet == "_Fix":
    changeRF=False 
else: 
    changeRF=True
if explainerName == "_SHAP_":
    XAI_v = SHAP(clfs,column_names,X_train,X_train_mod,X_test,protected,changeReferenceSet=changeRF)
else: 
    XAI_v = LIME(clfs,column_names,X_train,X_train_mod,X_test,protected,changeReferenceSet=changeRF)
XAISum,XAIMean = deltas(XAI_v,column_names)
resXAI = buildFinalResXAI(XAISum,XAIMean,protected,non_sensitive)
resXAI.to_csv(path_or_buf=path_res+samplingName+clfName+explainerName+XAISet+'.csv')
resXAI

In [ ]:
PIK = samplingName+clfName+explainerName+XAISet+".dat"
with open(path_res+"/"+PIK, "wb") as f:
  pickle.dump(XAI_v, f)

In [ ]:
PIK = samplingName+clfName+explainerName+XAISet+".dat"
with open(path_res+"/"+PIK, "rb") as f:
  XAI_v = pickle.load(f)

In [ ]:
XAISum,XAIMean = deltas(XAI_v,column_names)
XAIMean

In [ ]:
EValues = computeE(XAI_v,column_names)
EValues 

In [ ]:
XAIPlots(XAI_v,column_names,diffClfs=True)

In [ ]:
XAIPlots(XAI_v,protected,diffClfs=True)

### Comparison

In [ ]:
XAISet = "_Fix"
PIK = samplingName+clfName+explainerName+XAISet+'.dat"
with open(path_res+"/"+PIK, "rb") as f:
  XAI_v_fix = pickle.load(f)

XAISet = "_Change"
PIK = samplingName+clfName+explainerName+XAISet+'.dat"
with open(path_res+"/"+PIK, "rb") as f:
  XAI_v_change = pickle.load(f)

XAISum,XAIMean = deltas(XAI_v_fix,column_names,False,XAI_v_change)
XAIMean

In [ ]:
resXAI = buildFinalResXAI(XAISum,XAIMean,protected,non_sensitive)
resXAI = resXAI.round(3)
resXAI

In [ ]:
XAIPlots(XAI_v_fix,protected,True,XAI_v_change)